### How to use this notebook (for anyone not familiar with jupyter)

Before we get started, I'd like to note that while reading this notebook on github is fine, there are several interactive elements used to demonstrate several ideas that only render when the notebook is ran in a live kernel. This also allows you to play with the code samples inline with the rest of the notebook.

For the best experience I'd recommend installing [jupyter](https://jupyter.org/install) on your machine and cloning this repository so that you can run the notebooks locally. 

If however you'd prefer not to install anything, you can also run this notebook on Google's Colab [here](https://colab.research.google.com/github/jd-13/dsp-math-for-audio/blob/master/03_Envelope-Followers-Part-2.ipynb).

Once running click `Kernel` -> `Restart & Run All` (or `Runtime` -> `Run all` if you're using Colab) to render all of the interactive examples, and you are then free to edit the code and run individual cells as you wish.

In [1]:
"""
This code cell contains a few imports and functions that we'll need later but aren't of much interest.
The main content starts immediately after.
"""

from ipywidgets import interact
import matplotlib.pylab as plt
import math
import numpy as np
import scipy.signal
%matplotlib inline

# We'll use this to define the x-axis of any graphs we plot
NUM_SAMPLES = 500
SAMPLE_RANGE = np.linspace(0, NUM_SAMPLES, NUM_SAMPLES)

# Run at a lower than usual sample rate to keep the interactive examples from
# needing too much compute
SAMPLE_RATE = 10000

Y_LIMIT = 1.2

# Set the control limits
FREQUENCY_RANGE=(50, 1000, 100)
CUTOFF_RANGE=(6, 40, 2)

def sineWaveDecay(frequencyHz):
    """
    Returns a np array of length numSamples containing a sine wave impulse
    which decays linearly.
    """    
    # Build a saw that is half the length of the buffer
    amplitudeMap = (-scipy.signal.sawtooth(0.5 * np.linspace(0, 4 * np.pi, int(NUM_SAMPLES / 2))) + 1) / 2
    
    # Concatenate zeros either side of the saw
    zerosQuarter = np.zeros((int(NUM_SAMPLES / 4)))
    amplitudeMap = np.concatenate((zerosQuarter, amplitudeMap, zerosQuarter))
    
    # Smooth the amplitude map
    b, a = scipy.signal.butter(4, 0.04, btype="lowpass", analog=False)
    amplitudeMap = scipy.signal.lfilter(b, a, amplitudeMap)
    
    # Produce the sine wave, modulating the amplitude using the saw wave
    buffer = np.sin((2 * math.pi * frequencyHz) * (SAMPLE_RANGE / SAMPLE_RATE)) * amplitudeMap
    
    return buffer

def realSquareLawEnvelope(inputSignal, filterFunc):
    """
    Example implementation of a real square law envelope follower.
    
    Given an input signal and attack/release times in milliseconds,
    produces an envelope.
    """
    
    ## Step 1: Square the input
    rectifiedBuffer = inputSignal ** 2
    
    ## Step 2: Apply the filtering    
    envelopeBuffer = np.copy(rectifiedBuffer)            
    envelopeBuffer = filterFunc(envelopeBuffer)

    ## Step 3: Square root the output of the filter
    envelopeBuffer = np.sqrt(envelopeBuffer)

    return rectifiedBuffer, envelopeBuffer

## Intro

In the [previous notebook](https://github.com/jd-13/dsp-math-for-audio) we looked at how several different types of envelope filter work, and concluded that most of them use a rectification stage of some sort followed by a low pass filter. The content of the previous notebook was focused mostly on the performance impacts of using different rectification stages, but what about the filter stage?

In this notebook we'll look at how changing the filter stage of an envelope follower affects its performance, and for consistency we'll use an envelope follower with a real square law rectification stage throughout the examples in this notebook.

First, let's review how a real square law envelope follower works:

![](https://raw.githubusercontent.com/jd-13/dsp-math-for-audio/master/assets/envelope-followers/realSquareLaw.png)

In any envelope follower like the one above, the frequency response of the filter being used determines the responsiveness of the envelope follower, often described as the attack and release time.

As you may have noticed from the code in the previous notebook, when controlling the attack and release times we're actually controlling the coefficients of the filter, therefore changing it's frequency response.

Attenuating higher frequencies more results in a smoother but slower response, while passing through more high frequencies creates a faster response with more ripple. Now, let's look at a demonstration of this using a real square law envelope follower.

This example is the same as we have seen from the end of the previous notebook, however the attack and release controls have been replaced with direct control over the cutoff frequency of the filter. You can also control the frequency of the input signal to see how the envelope follower responds to different frequencies.

In [2]:
class ButterworthLP:    
    def __init__(self, order, cutoffHz):
        angularCutoff = (2 * np.pi * cutoffHz) / (0.5 * SAMPLE_RATE)
        self._b, self._a = scipy.signal.butter(order, angularCutoff, "lowpass", False, "ba")
        
    def __call__(self, inputBuffer):
        return scipy.signal.lfilter(self._b, self._a, inputBuffer)
    
@interact(inputSineFrequency=(50, 1000, 100), cutoff=(1, 100, 10))
def plotRealSquareLaw(inputSineFrequency, cutoff):
    """
    Feeds a sine wave impluse to the envelope follower and plots the outputs.
    """
    
    # Generate the input signal
    signalBuffer = sineWaveDecay(inputSineFrequency)  
    
    # Generate the output from the envelope at its thresholding stage and its
    # final output
    rectifiedBuffer, envelopeBuffer = realSquareLawEnvelope(signalBuffer, ButterworthLP(1, cutoff))

    # Plot the envelope outputs
    plt.figure(figsize=(15, 4))

    plt.plot(SAMPLE_RANGE, signalBuffer)
    plt.plot(SAMPLE_RANGE, envelopeBuffer)

    plt.ylim(bottom=-Y_LIMIT, top=Y_LIMIT)
    plt.title("Envelope Output")
    plt.xlabel("Samples")
    plt.ylabel("Displacement")

    plt.show()

interactive(children=(IntSlider(value=450, description='inputSineFrequency', max=1000, min=50, step=100), IntSlider(value=41, description='cutoff', min=1, step=10), Output()), _dom_classes=('widget-interact',))

Notice how a higher cutoff frequency causes the envelope follower to track the peaks and troughs of the wave more closely, while a lower cutoff frequency causes the follower to produce a smoother output.

## The effects of a filter's order

So what about changing the order of the filter? A higher order filter would attenuate high frequencies more than a lower order filter with the same cutoff frequency, so would be expected to result in a slower and smoother response.

The below example compares a first, second, and third order Butterworth at the same cutoff frequency. You can use the sliders to control the cutoff frequency which the filters are set to and the frequency of the input signal.

In [3]:
@interact(inputSineFrequency=FREQUENCY_RANGE, cutoff=CUTOFF_RANGE)
def plotComparison1(inputSineFrequency, cutoff):
    """
    Feeds a sine wave impluse to the envelope followers and plots the outputs.
    """
    
    # Generate the input signal
    signalBuffer = sineWaveDecay(inputSineFrequency)
        
    _, firstOrderBuffer = realSquareLawEnvelope(signalBuffer, ButterworthLP(1, cutoff))
    _, secondOrderBuffer = realSquareLawEnvelope(signalBuffer, ButterworthLP(2, cutoff))
    _, thirdOrderBuffer = realSquareLawEnvelope(signalBuffer, ButterworthLP(3, cutoff))

    plt.figure(figsize=(15, 4))

    plt.plot(SAMPLE_RANGE, signalBuffer, label="Input")
    plt.plot(SAMPLE_RANGE, firstOrderBuffer, label="First Order")
    plt.plot(SAMPLE_RANGE, secondOrderBuffer, label="Second Order")
    plt.plot(SAMPLE_RANGE, thirdOrderBuffer, label="Third Order")

    plt.ylim(bottom=-Y_LIMIT, top=Y_LIMIT)
    plt.xlabel("Samples")
    plt.ylabel("Displacement")
    plt.legend(loc='upper left')

    plt.show()

interactive(children=(IntSlider(value=450, description='inputSineFrequency', max=1000, min=50, step=100), IntSlider(value=22, description='cutoff', max=40, min=6, step=2), Output()), _dom_classes=('widget-interact',))

As expected, we can see that the envelope follower built from second and third order filters lag behind the one with a first order filter, but they also show much less ripple.

You might now be wondering if lowering the cutoff frequency of the first and second order filters by some value results in the same performance as the third order filter, we can see the result of this experiment below.

In [4]:
@interact(inputSineFrequency=FREQUENCY_RANGE)
def plotComparison2(inputSineFrequency):
    """
    Feeds a sine wave impluse to the envelope followers and plots the outputs.
    """
    
    # Generate the input signal
    signalBuffer = sineWaveDecay(inputSineFrequency)
    
    CUTOFF = 20
    
    _, firstOrderBuffer = realSquareLawEnvelope(signalBuffer, ButterworthLP(1, CUTOFF-18))
    _, secondOrderBuffer = realSquareLawEnvelope(signalBuffer, ButterworthLP(2, CUTOFF-10))
    _, thirdOrderBuffer = realSquareLawEnvelope(signalBuffer, ButterworthLP(3, CUTOFF))

    plt.figure(figsize=(15, 4))

    plt.plot(SAMPLE_RANGE, signalBuffer, label="Input")
    plt.plot(SAMPLE_RANGE, firstOrderBuffer, label="First Order")
    plt.plot(SAMPLE_RANGE, secondOrderBuffer, label="Second Order")
    plt.plot(SAMPLE_RANGE, thirdOrderBuffer, label="Third Order")

    plt.ylim(bottom=-Y_LIMIT, top=Y_LIMIT)
    plt.xlabel("Samples")
    plt.ylabel("Displacement")
    plt.legend(loc='upper left')

    plt.show()

interactive(children=(IntSlider(value=450, description='inputSineFrequency', max=1000, min=50, step=100), Output()), _dom_classes=('widget-interact',))

We can see here that once the cutoff frequency of the first and second order filters has been lowered sufficiently to push their attack times back to that of the third order filter, they have a much lower peak amplitude, slightly more ripple, and a longer release time (*much* longer in the case of the first order filter) than the envelope follower built from a third order filter - so using a higher order filter can result in a very different behaviour that can't be achieved by simply changing the cutoff frequency of a lower order filter.

## The effects of a filter's architecture

So far we've only used Butterworth filters since they're very common, but now let's see how an envelope follower performs when using a different architecture. If you're not familiar with different filter types, the Intro section of the [IIR Filters notebook](https://github.com/jd-13/dsp-math-for-audio) has a interactive example to summarise.

Compared to the Butterworth, the Chebyshev filter has a steeper slope between its passband and stopband, so we might expect the Chebyshev to behave a little like the higher order Butterworth, ie. a slower attack and less ripple. An important difference is the Butterworth filter has a smooth response where the Chebyshev filter has ripple in the passband.

The elliptic filter has an even steeper slope than the Chebyshev, but also has ripple in both the passband and the stopband, so we might expect an envelope follower built from an elliptic filter to look like a more extreme version of the Chebyshev.

The Bessel filter is distinguished by having a linear phase response and flat group delay, but the cost of this is a flatter slope between its passband and stopband. We might not expect phase response or group delay to affect the ability of the envelope follower to follow a signal, but the flatter slope very likely will.

In the code below, we set up pairs of first and third order filters with the same cutoff frequency. The tick boxes can be used to compare envelope followers built from different filter types. As before, the input signal frequency and the filters' cutoff frequency can both be controlled using the sliders.

In [5]:
# Below are a few classes that handle the filtering, docstrings and formatting omitted for brevity
class ChebyshevLP:
    def __init__(self, order, cutoffHz):
        angularCutoff = (2 * np.pi * cutoffHz) / (0.5 * SAMPLE_RATE)
        self._b, self._a = scipy.signal.cheby1(order, 3, angularCutoff, "lowpass", False, "ba")
        
    def __call__(self, inputBuffer):
        return scipy.signal.lfilter(self._b, self._a, inputBuffer)
    
class EllipticLP:
    def __init__(self, order, cutoffHz):
        angularCutoff = (2 * np.pi * cutoffHz) / (0.5 * SAMPLE_RATE)
        self._b, self._a = scipy.signal.ellip(order, 3, 20, angularCutoff, "lowpass", False, "ba")
        
    def __call__(self, inputBuffer):
        return scipy.signal.lfilter(self._b, self._a, inputBuffer)
    
class BesselLP:    
    def __init__(self, order, cutoffHz):
        angularCutoff = (2 * np.pi * cutoffHz) / (0.5 * SAMPLE_RATE)
        self._b, self._a = scipy.signal.bessel(order, angularCutoff, "lowpass", False)
        
    def __call__(self, inputBuffer):
        return scipy.signal.lfilter(self._b, self._a, inputBuffer)

@interact(inputSineFrequency=FREQUENCY_RANGE, cutoff=CUTOFF_RANGE, butterworth=True, chebyshev=True, elliptic=False, bessel=False)
def plotComparison3(inputSineFrequency, cutoff, butterworth, chebyshev, elliptic, bessel):
    """
    Feeds a sine wave impluse to the envelope followers and plots the outputs.
    """
    
    # Generate the input signal
    signalBuffer = sineWaveDecay(inputSineFrequency)

    # Plot the input signal
    plt.figure(figsize=(15, 4))
    plt.plot(SAMPLE_RANGE, signalBuffer, label="Input")

    # Plot each envelope follower that is selected 
    filterMapping = {
        "Butterworth": [butterworth, ButterworthLP],
        "Chebyshev": [chebyshev, ChebyshevLP],
        "Elliptic": [elliptic, EllipticLP],
        "Bessel": [bessel, BesselLP]
    }

    for filterClass in filterMapping.items():
        filterName = filterClass[0]
        filterActive = filterClass[1][0]
        filterFunc = filterClass[1][1]
        
        if filterActive:
            _, firstOrderBuffer = realSquareLawEnvelope(signalBuffer, filterFunc(1, cutoff))
            _, thirdOrderBuffer = realSquareLawEnvelope(signalBuffer, filterFunc(3, cutoff))

            plt.plot(SAMPLE_RANGE, firstOrderBuffer, label=f"First Order {filterName}")
            plt.plot(SAMPLE_RANGE, thirdOrderBuffer, label=f"Third Order {filterName}")

    # Configure the plot
    plt.ylim(bottom=-Y_LIMIT, top=Y_LIMIT)
    plt.xlabel("Samples")
    plt.ylabel("Displacement")
    plt.legend(loc='upper left')

    plt.show()

interactive(children=(IntSlider(value=450, description='inputSineFrequency', max=1000, min=50, step=100), IntSlider(value=22, description='cutoff', max=40, min=6, step=2), Checkbox(value=True, description='butterworth'), Checkbox(value=True, description='chebyshev'), Checkbox(value=False, description='elliptic'), Checkbox(value=False, description='bessel'), Output()), _dom_classes=('widget-interact',))

Let's compare the behaviours we can see in the above example:

### Chebyshev
The envelope followers built from the first order Butterworth and Chebyshev both behave identically, but the third order ones are a little different. The Chebyshev produces a longer attack and release time as expected, but actually introduces more ripple due to the ripple in the Chebyshev's passband.

### Elliptic
Again, the envelope follower built from a first order elliptic filter behaves the same as the first order Butterworth and Chebyshev, but the third order filter is more interesting. It shows a lot more ripple, and actually has a slightly faster attack than the Chebyshev or Butterworth - possibly because the ripple in the stop band actually reduces its attenuation of high frequencies.

### Bessel
As expected, an envelope follower built from a Bessel filter behaves very similarly to the Butterworth, with the third order variant having just a slightly slower response and a slightly smaller peak.

## Conclusion
This was just a quick introduction to how your choice of filter can affect the way your envelope follower behaves. We've seen from the demonstrations that both the order and type of an envelope follower's filter stage are important factors, with a general rule being:

    More attenuation of high frequencies = slower response, less ripple

## Further Reading
Below are several resources that I found useful while researching this notebook. If you wish to read more about this topic, these may be good places to start.

[Envelope detector](https://en.wikipedia.org/wiki/Envelope_detector)  
[Digital Envelope Detection: The Good, the Bad, and the Ugly](https://www.dsprelated.com/showarticle/938.php)  
[Meaning of Hilbert Transform](https://dsp.stackexchange.com/questions/25845/meaning-of-hilbert-transform)  
[Envelope detector](http://www.musicdsp.org/en/latest/Analysis/97-envelope-detector.html)  
[Envelope Controlled Filters](http://elliott-randall.com/2011/06/envelope-controlled-filters/)  
[Auto-wah](https://en.wikipedia.org/wiki/Auto-wah)  
[Amplitude Modulation](https://user.eng.umd.edu/~tretter/commlab/c6713slides/ch5.pdf)